# Periodic download of IEX stock-ticker data

## Read IEX API credentials from `~/.config/iex.ini`

In [1]:
from pathlib import Path
config_path = Path.home() / '.config' / 'iex.ini'

from configparser import ConfigParser
config = ConfigParser()
config.read(str(config_path))
iex_config = config['iex']

api = 'https://cloud.iexapis.com'
public_key = iex_config['public_key']
secret_key = iex_config['secret_key']

In [2]:
tickers = sorted("MMM ABT ABBV ABMD ACN ATVI ADBE AMD AAP AES AMG AFL A APD AKAM ALK ALB ARE ALXN ALGN ALLE AGN ADS LNT ALL GOOGL GOOG MO AMZN AMCR AEE AAL AEP AXP AIG AMT AWK AMP ABC AME AMGN APH ADI ANSS ANTM AON AOS APA AIV AAPL AMAT APTV ADM ARNC ANET AJG AIZ ATO T ADSK ADP AZO AVB AVY BKR BLL BAC BK BAX BBT BDX BRK.B BBY BIIB BLK HRB BA BKNG BWA BXP BSX BMY AVGO BR BF.B CHRW COG CDNS CPB COF CPRI CAH KMX CCL CAT CBOE CBRE CBS CDW CE CELG CNC CNP CTL CERN CF SCHW CHTR CVX CMG CB CHD CI XEC CINF CTAS CSCO C CFG CTXS CLX CME CMS KO CTSH CL CMCSA CMA CAG CXO COP ED STZ COO CPRT GLW CTVA COST COTY CCI CSX CMI CVS DHI DHR DRI DVA DE DAL XRAY DVN FANG DLR DFS DISCA DISCK DISH DG DLTR D DOV DOW DTE DUK DRE DD DXC ETFC EMN ETN EBAY ECL EIX EW EA EMR ETR EOG EFX EQIX EQR ESS EL EVRG ES RE EXC EXPE EXPD EXR XOM FFIV FB FAST FRT FDX FIS FITB FE FRC FISV FLT FLIR FLS FMC F FTNT FTV FBHS FOXA FOX BEN FCX GPS GRMN IT GD GE GIS GM GPC GILD GL GPN GS GWW HAL HBI HOG HIG HAS HCA HCP HP HSIC HSY HES HPE HLT HFC HOLX HD HON HRL HST HPQ HUM HBAN HII IEX IDXX INFO ITW ILMN IR INTC ICE IBM INCY IP IPG IFF INTU ISRG IVZ IPGP IQV IRM JKHY JEC JBHT SJM JNJ JCI JPM JNPR KSU K KEY KEYS KMB KIM KMI KLAC KSS KHC KR LB LHX LH LRCX LW LVS LEG LDOS LEN LLY LNC LIN LKQ LMT L LOW LYB MTB MAC M MRO MPC MKTX MAR MMC MLM MAS MA MKC MXIM MCD MCK MDT MRK MET MTD MGM MCHP MU MSFT MAA MHK TAP MDLZ MNST MCO MS MOS MSI MSCI MYL NDAQ NOV NTAP NFLX NWL NEM NWSA NWS NEE NLSN NKE NI NBL JWN NSC NTRS NOC NCLH NRG NUE NVDA NVR ORLY OXY OMC OKE ORCL PCAR PKG PH PAYX PYPL PNR PBCT PEP PKI PRGO PFE PM PSX PNW PXD PNC PPG PPL PFG PG PGR PLD PRU PEG PSA PHM PVH QRVO PWR QCOM DGX RL RJF RTN O REG REGN RF RSG RMD RHI ROK ROL ROP ROST RCL CRM SBAC SLB STX SEE SRE SHW SPG SWKS SLG SNA SO LUV SPGI SWK SBUX STT SYK STI SIVB SYMC SYF SNPS SYY TMUS TROW TTWO TPR TGT TEL FTI TFX TXN TXT TMO TIF TWTR TJX TSCO TDG TRV TRIP TSN UDR ULTA USB UAA UA UNP UAL UNH UPS URI UTX UHS UNM VFC VLO VAR VTR VRSN VRSK VZ VRTX VIAB V VNO VMC WAB WMT WBA DIS WM WAT WEC WCG WFC WELL WDC WU WRK WY WHR WMB WLTW WYNN XEL XRX XLNX XYL YUM ZBH ZION ZTS".split(" "))
num_tickers = len(tickers)
num_tickers

505

In [33]:
aapl = tickers.index('AAPL'); aapl

3

In [3]:
from datetime import datetime
from dateutil.parser import parse
from datetime import timedelta as Δ

time = datetime.now
now = time()
today = now.date()
today

datetime.date(2019, 10, 30)

In [4]:
data_dir = Path.cwd() / 'data'
data_dir.mkdir(parents=True, exist_ok=True)

In [5]:
from sys import executable as python
!{python} -m pip install -Uq requests
from requests import get as GET

In [6]:
import json

def fetch(date_str, ticker):
    out_path = data_dir / ('%s-%s' % (date_str, ticker))
    if out_path.exists():
        return True

    print('Fetching data for %s from %s' % (ticker, date_str))

    url = f'https://cloud.iexapis.com/stable/stock/{ticker}/chart/date/{date_str}?token={secret_key}'
    resp = GET(url)
    resp.raise_for_status()
    with out_path.open('wb') as f:
        f.write(resp.content)

    data = json.loads(resp.content)
    if data:
        return True

    return False

In [7]:
%%time
from concurrent.futures import ThreadPoolExecutor

end_date = today
start_date = datetime(2019, 8, 1).date()
N = 32

def get_dates(start_date, end_date, step=1):
    date = start_date
    while date != end_date:
        if date.weekday() <= 4:
            yield date
        date += Δ(days=step)

dates = list(get_dates(start_date, end_date))

for date in dates:
    date_str = date.strftime('%Y%m%d')
    
    with ThreadPoolExecutor(max_workers = N) as p:
        results = p.map(lambda ticker: fetch(date_str, ticker), tickers)
    
    found_data = True in results
    if not found_data:
        print('No data found for %s; breaking' % date)

CPU times: user 1.22 s, sys: 387 ms, total: 1.61 s
Wall time: 1.36 s


In [8]:
!{python} -m pip install -Uq pandas
from pandas import DataFrame as DF, read_csv, read_json
import pandas as pd

In [9]:
minutes = 390  # [9:30am,4:00pm)

In [10]:
features = [ 'open', 'close', 'high', 'low', 'average', 'volume', 'notional', 'numberOfTrades' ]
cols = [ 'datetime', 'ticker' ] + features

In [11]:
def load_data_arr(date, ticker):
    date_str = date.strftime('%Y%m%d')
    out_path = data_dir / ('%s-%s' % (date_str, ticker))
    if not out_path.exists():
        arr = zeros((minutes, len(features)))
        arr[:] = nan
        return arr
    df = read_json(out_path)
    if df.empty:
        arr = zeros((minutes, len(features)))
        arr[:] = nan
        return arr
    arr = df[features].values
    assert arr.shape == (minutes, len(features))
    return arr

In [12]:
!{python} -m pip install -Uq numpy
import numpy as np
from numpy import array, nan, zeros

In [13]:
def load_date_arr(date):
    arr = array([ 
        load_data_arr(start_date, ticker) 
        for ticker in tickers 
    ]) \
    .reshape((
        minutes, 
        len(tickers), 
        len(features),
    ))
    assert arr.shape == (minutes, num_tickers, len(features))
    return arr

In [14]:
!{python} -m pip install -Uq joblib
from joblib import Parallel, delayed

In [15]:
all = \
    array(
        Parallel(n_jobs=8)(delayed(load_date_arr)(date) for date in dates)
    )

In [195]:
shape[0]

24960

In [16]:
shape =  all.shape; shape

(64, 390, 505, 8)

In [17]:
all = all.reshape(shape[0] * shape[1], *shape[2:]); all.shape

(24960, 505, 8)

In [24]:
np.save(str(data_dir / 'all.npy'), all)

In [28]:
shape = all.shape
all = all.reshape(shape[:-2] + ((shape[-2] * shape[-1],)))
shape = all.shape
shape

(24960, 4040)

In [193]:
np.count_nonzero(np.isnan(all[:, aapl_avg]))

9792

In [200]:
all = all[~np.isnan(all[:, aapl_avg])]
shape = all.shape; shape

(15168, 4040)

In [202]:
aapl_avg = aapl * len(features) + 4
out = all[:, aapl_avg]
out = np.roll(out, -1)[:-1]
all = all[:-1]
shape = all.shape
out.shape, out

((15167,), array([ 91.835, 204.915,  85.943, ...,  61.1  ,  54.221, 138.386]))

In [63]:
from numpy.random import randint, shuffle
from numpy import arange, nan_to_num

In [203]:
time_steps = 20

def make_train_val_idxs(train=None, val=None):
    num_samples = shape[0] - time_steps

    if train is None and val is None:
        raise Exception('Specify at least one of {train, val}')
            
    if train is not None and train < 1:
        train = int(train * num_samples)

    if val is not None and val < 1:
        val = int(train * num_samples)
    
    if val is None:
        val = num_samples - train
    
    if train is None:
        train = num_samples - val
    
    if train + val > num_samples:
        raise Exception('%d + %d > %d' % (train, val, num_samples))

    sample_idxs = arange(time_steps, shape[0])
    shuffle(sample_idxs)
    training_idxs = sample_idxs[:train]
    validation_idxs = sample_idxs[-val:]
    return (training_idxs, validation_idxs)

def make_train_val_sets(train=None, val=None):
    train_idxs, val_idxs = make_train_val_idxs(train, val)
    train_x = nan_to_num(array([ all[(idx - time_steps):idx] for idx in train_idxs ]))
    train_y = nan_to_num(array([ all[idx][aapl_avg] for idx in train_idxs ]))
    val_x = nan_to_num(array([ all[(idx - time_steps):idx] for idx in val_idxs ]))
    val_y = nan_to_num(array([ all[idx][aapl_avg] for idx in val_idxs ]))
    return (
        train_x,
        train_y,
        val_x,
        val_y,
    )

In [204]:
train_x, train_y, val_x, val_y = make_train_val_sets(12000, 1000)
[ a.shape for a in [train_x, train_y, val_x, val_y]]

[(12000, 20, 4040), (12000,), (1000, 20, 4040), (1000,)]

In [205]:
shape[0]

15167

In [19]:
!{python} -m pip install -Uq tensorflow
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN

In [206]:
model = Sequential([
    SimpleRNN(32, input_shape=(time_steps, num_tickers * len(features))),
    Dense(1),
])

In [207]:
model.build()

In [208]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_7 (SimpleRNN)     (None, 32)                130336    
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 33        
Total params: 130,369
Trainable params: 130,369
Non-trainable params: 0
_________________________________________________________________


In [209]:
model.compile(loss='mse', optimizer='sgd')

In [210]:
model.fit(train_x, train_y, 
          #validation_data=(val_x, val_y),
          batch_size=40,
          epochs=10)

Train on 12000 samples
Epoch 1/10
12000/12000 [==============================] - 18s 1ms/sample - loss: 16344.0626
Epoch 2/10
12000/12000 [==============================] - 14s 1ms/sample - loss: 15546.2652
Epoch 3/10
12000/12000 [==============================] - 14s 1ms/sample - loss: 15530.9598
Epoch 4/10
12000/12000 [==============================] - 14s 1ms/sample - loss: 15515.1891
Epoch 5/10
12000/12000 [==============================] - 15s 1ms/sample - loss: 15502.5735
Epoch 6/10
12000/12000 [==============================] - 14s 1ms/sample - loss: 15523.4275
Epoch 7/10
12000/12000 [==============================] - 14s 1ms/sample - loss: 15517.6569
Epoch 8/10
12000/12000 [==============================] - 14s 1ms/sample - loss: 15560.4271
Epoch 9/10
12000/12000 [==============================] - 16s 1ms/sample - loss: 15538.1058
Epoch 10/10
12000/12000 [==============================] - 14s 1ms/sample - loss: 15486.1096


In [211]:
model.fit(train_x, train_y, 
          #validation_data=(val_x, val_y),
          batch_size=40,
          epochs=20)

Train on 12000 samples
Epoch 1/20
12000/12000 [==============================] - 15s 1ms/sample - loss: 15545.4982
Epoch 2/20
12000/12000 [==============================] - 14s 1ms/sample - loss: 15530.5658
Epoch 3/20
12000/12000 [==============================] - 14s 1ms/sample - loss: 15478.8371
Epoch 4/20
12000/12000 [==============================] - 14s 1ms/sample - loss: 15528.6299
Epoch 5/20
12000/12000 [==============================] - 14s 1ms/sample - loss: 15508.9193
Epoch 6/20
12000/12000 [==============================] - 14s 1ms/sample - loss: 15532.5590
Epoch 7/20
12000/12000 [==============================] - 14s 1ms/sample - loss: 15535.9591
Epoch 8/20
12000/12000 [==============================] - 14s 1ms/sample - loss: 15509.6757
Epoch 9/20
12000/12000 [==============================] - 14s 1ms/sample - loss: 15512.1479
Epoch 10/20
12000/12000 [==============================] - 14s 1ms/sample - loss: 15517.8358
Epoch 11/20
12000/12000 [==============================]

In [212]:
model.compile(loss='mse', optimizer='adam')

In [213]:
model.fit(train_x, train_y, 
          validation_data=(val_x, val_y),
          batch_size=40,
          epochs=20)

Train on 12000 samples, validate on 1000 samples
Epoch 1/20
12000/12000 [==============================] - 19s 2ms/sample - loss: 15419.7773 - val_loss: 15587.0837
Epoch 2/20
12000/12000 [==============================] - 16s 1ms/sample - loss: 15412.6172 - val_loss: 15572.2574
Epoch 3/20
12000/12000 [==============================] - 16s 1ms/sample - loss: 15406.8898 - val_loss: 15560.0918
Epoch 4/20
12000/12000 [==============================] - 15s 1ms/sample - loss: 15402.4290 - val_loss: 15549.7905
Epoch 5/20
12000/12000 [==============================] - 16s 1ms/sample - loss: 15398.7806 - val_loss: 15541.3923
Epoch 6/20
12000/12000 [==============================] - 15s 1ms/sample - loss: 15395.8807 - val_loss: 15533.2663
Epoch 7/20
12000/12000 [==============================] - 15s 1ms/sample - loss: 15393.4201 - val_loss: 15527.3017
Epoch 8/20
12000/12000 [==============================] - 15s 1ms/sample - loss: 15391.4413 - val_loss: 15521.6040
Epoch 9/20
12000/12000 [=======

In [214]:
model.fit(train_x, train_y, 
          validation_data=(val_x, val_y),
          batch_size=40,
          epochs=20)

Train on 12000 samples, validate on 1000 samples
Epoch 1/20
12000/12000 [==============================] - 15s 1ms/sample - loss: 15379.3563 - val_loss: 15491.4467
Epoch 2/20
12000/12000 [==============================] - 17s 1ms/sample - loss: 15378.7538 - val_loss: 15490.7604
Epoch 3/20
12000/12000 [==============================] - 16s 1ms/sample - loss: 15378.3939 - val_loss: 15489.9805
Epoch 4/20
12000/12000 [==============================] - 15s 1ms/sample - loss: 15377.8064 - val_loss: 15489.4879
Epoch 5/20
12000/12000 [==============================] - 16s 1ms/sample - loss: 15377.2507 - val_loss: 15488.4293
Epoch 6/20
12000/12000 [==============================] - 16s 1ms/sample - loss: 15376.9178 - val_loss: 15488.1877
Epoch 7/20
12000/12000 [==============================] - 15s 1ms/sample - loss: 15376.4122 - val_loss: 15488.0419
Epoch 8/20
12000/12000 [==============================] - 15s 1ms/sample - loss: 15375.9355 - val_loss: 15487.4029
Epoch 9/20
12000/12000 [=======

In [215]:
model.fit(train_x, train_y, 
          validation_data=(val_x, val_y),
          batch_size=40,
          epochs=200)

Train on 12000 samples, validate on 1000 samples
Epoch 1/200
12000/12000 [==============================] - 17s 1ms/sample - loss: 15371.6789 - val_loss: 15484.1590
Epoch 2/200
12000/12000 [==============================] - 16s 1ms/sample - loss: 15371.4278 - val_loss: 15484.0324
Epoch 3/200
12000/12000 [==============================] - 15s 1ms/sample - loss: 15371.1502 - val_loss: 15484.1951
Epoch 4/200
12000/12000 [==============================] - 15s 1ms/sample - loss: 15370.8957 - val_loss: 15483.6107
Epoch 5/200
12000/12000 [==============================] - 15s 1ms/sample - loss: 15370.7880 - val_loss: 15483.7685
Epoch 6/200
12000/12000 [==============================] - 15s 1ms/sample - loss: 15370.4712 - val_loss: 15483.7804
Epoch 7/200
12000/12000 [==============================] - 15s 1ms/sample - loss: 15370.2009 - val_loss: 15484.2235
Epoch 8/200
12000/12000 [==============================] - 15s 1ms/sample - loss: 15370.0815 - val_loss: 15483.6573
Epoch 9/200
12000/12000

12000/12000 [==============================] - 15s 1ms/sample - loss: 15364.4567 - val_loss: 15487.8361
Epoch 71/200
12000/12000 [==============================] - 15s 1ms/sample - loss: 15364.4859 - val_loss: 15488.8021
Epoch 72/200
12000/12000 [==============================] - 16s 1ms/sample - loss: 15364.5323 - val_loss: 15488.8044
Epoch 73/200
12000/12000 [==============================] - 17s 1ms/sample - loss: 15364.4228 - val_loss: 15488.7661
Epoch 74/200
12000/12000 [==============================] - 15s 1ms/sample - loss: 15364.4240 - val_loss: 15489.0221
Epoch 75/200
12000/12000 [==============================] - 15s 1ms/sample - loss: 15364.3305 - val_loss: 15488.6086
Epoch 76/200
12000/12000 [==============================] - 18s 2ms/sample - loss: 15364.4042 - val_loss: 15489.1560
Epoch 77/200
12000/12000 [==============================] - 18s 1ms/sample - loss: 15364.1848 - val_loss: 15489.3888
Epoch 78/200
12000/12000 [==============================] - 16s 1ms/sample - 

Epoch 140/200
12000/12000 [==============================] - 16s 1ms/sample - loss: 15363.4934 - val_loss: 15494.7523
Epoch 141/200
12000/12000 [==============================] - 15s 1ms/sample - loss: 15363.3948 - val_loss: 15494.3557
Epoch 142/200
12000/12000 [==============================] - 15s 1ms/sample - loss: 15363.4620 - val_loss: 15495.2337
Epoch 143/200
12000/12000 [==============================] - 15s 1ms/sample - loss: 15363.4224 - val_loss: 15494.8069
Epoch 144/200
12000/12000 [==============================] - 17s 1ms/sample - loss: 15363.4144 - val_loss: 15494.9970
Epoch 145/200
12000/12000 [==============================] - 17s 1ms/sample - loss: 15363.4223 - val_loss: 15495.0199
Epoch 146/200
12000/12000 [==============================] - 16s 1ms/sample - loss: 15363.4744 - val_loss: 15495.5698
Epoch 147/200
12000/12000 [==============================] - 16s 1ms/sample - loss: 15363.3751 - val_loss: 15494.8303
Epoch 148/200
12000/12000 [=============================

In [176]:
model.fit(train_x, train_y, 
          batch_size=10,
          epochs=10)

Train on 10000 samples
Epoch 1/10
10000/10000 [==============================] - 15s 1ms/sample - loss: 12392.3727
Epoch 2/10
10000/10000 [==============================] - 14s 1ms/sample - loss: 12332.7983
Epoch 3/10
10000/10000 [==============================] - 16s 2ms/sample - loss: 12362.4786
Epoch 4/10
10000/10000 [==============================] - 15s 2ms/sample - loss: 12381.8192
Epoch 5/10
10000/10000 [==============================] - 15s 1ms/sample - loss: 12341.3580
Epoch 6/10
10000/10000 [==============================] - 15s 1ms/sample - loss: 12337.6320
Epoch 7/10
10000/10000 [==============================] - 15s 1ms/sample - loss: 12358.0993
Epoch 8/10
10000/10000 [==============================] - 15s 1ms/sample - loss: 12322.7205
Epoch 9/10
10000/10000 [==============================] - 15s 1ms/sample - loss: 12390.4263
Epoch 10/10
10000/10000 [==============================] - 15s 2ms/sample - loss: 12381.1204


In [177]:
model.fit(train_x, train_y, 
          batch_size=10,
          epochs=20)

Train on 10000 samples
Epoch 1/20
10000/10000 [==============================] - 16s 2ms/sample - loss: 12356.6821
Epoch 2/20
10000/10000 [==============================] - 15s 1ms/sample - loss: 12356.3212
Epoch 3/20
10000/10000 [==============================] - 15s 1ms/sample - loss: 12393.1001
Epoch 4/20
10000/10000 [==============================] - 15s 2ms/sample - loss: 12219.4047
Epoch 5/20
10000/10000 [==============================] - 15s 2ms/sample - loss: 12299.5797
Epoch 6/20
10000/10000 [==============================] - 15s 2ms/sample - loss: 12324.0439
Epoch 7/20
10000/10000 [==============================] - 15s 2ms/sample - loss: 12323.7565
Epoch 8/20
10000/10000 [==============================] - 15s 2ms/sample - loss: 12325.0019
Epoch 9/20
10000/10000 [==============================] - 15s 1ms/sample - loss: 12394.5230
Epoch 10/20
10000/10000 [==============================] - 15s 2ms/sample - loss: 12326.8734
Epoch 11/20
10000/10000 [==============================]

In [141]:
from numpy.random import random

In [142]:
train_x = random((100,)).astype(np.float32)
train_y = random((100,)).astype(np.float32)
train_x, train_y

(array([0.46147427, 0.4936299 , 0.7725021 , 0.18668415, 0.19160269,
        0.5329875 , 0.89422435, 0.8431803 , 0.45767853, 0.9006393 ,
        0.30371124, 0.8654656 , 0.64627594, 0.2774724 , 0.34715426,
        0.7767715 , 0.08706174, 0.9893154 , 0.6400852 , 0.9627552 ,
        0.50067353, 0.56687176, 0.48472205, 0.6378169 , 0.6819773 ,
        0.6446982 , 0.13012254, 0.27104077, 0.50481087, 0.25702876,
        0.59998226, 0.45324805, 0.50154966, 0.43757737, 0.66555214,
        0.9135334 , 0.7003318 , 0.23003371, 0.05756523, 0.49323606,
        0.75259066, 0.6157169 , 0.6374104 , 0.5707051 , 0.6783405 ,
        0.89434326, 0.8651092 , 0.9301288 , 0.586712  , 0.7044457 ,
        0.7020338 , 0.09967529, 0.7703017 , 0.85118467, 0.14892976,
        0.24892834, 0.2034809 , 0.26588657, 0.05641429, 0.03270473,
        0.73204887, 0.9724218 , 0.31882647, 0.17593206, 0.7806721 ,
        0.96902066, 0.84500116, 0.7622705 , 0.00976924, 0.772007  ,
        0.31686476, 0.95740134, 0.16487914, 0.99

In [143]:
model = Sequential([
    SimpleRNN(10, input_shape=(10, 1)),
    Dense(1),
])

In [144]:
model.build()

In [145]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_4 (SimpleRNN)     (None, 10)                120       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 11        
Total params: 131
Trainable params: 131
Non-trainable params: 0
_________________________________________________________________


In [146]:
model.compile(loss='mse', optimizer='sgd')

In [151]:
train_x.reshape(10, 10, 1)

array([[[0.46147427],
        [0.4936299 ],
        [0.7725021 ],
        [0.18668415],
        [0.19160269],
        [0.5329875 ],
        [0.89422435],
        [0.8431803 ],
        [0.45767853],
        [0.9006393 ]],

       [[0.30371124],
        [0.8654656 ],
        [0.64627594],
        [0.2774724 ],
        [0.34715426],
        [0.7767715 ],
        [0.08706174],
        [0.9893154 ],
        [0.6400852 ],
        [0.9627552 ]],

       [[0.50067353],
        [0.56687176],
        [0.48472205],
        [0.6378169 ],
        [0.6819773 ],
        [0.6446982 ],
        [0.13012254],
        [0.27104077],
        [0.50481087],
        [0.25702876]],

       [[0.59998226],
        [0.45324805],
        [0.50154966],
        [0.43757737],
        [0.66555214],
        [0.9135334 ],
        [0.7003318 ],
        [0.23003371],
        [0.05756523],
        [0.49323606]],

       [[0.75259066],
        [0.6157169 ],
        [0.6374104 ],
        [0.5707051 ],
        [0.6783405 ],
  

In [154]:
model.fit(train_x.reshape(10, 10, 1) * 100, train_y[:10] * 100, 
          batch_size=10,
          epochs=10)

Train on 10 samples
Epoch 1/10
10/10 [==============================] - 0s 1ms/sample - loss: 1693.9922
Epoch 2/10
10/10 [==============================] - 0s 593us/sample - loss: 1245.9900
Epoch 3/10
10/10 [==============================] - 0s 475us/sample - loss: 963.2513
Epoch 4/10
10/10 [==============================] - 0s 502us/sample - loss: 778.3412
Epoch 5/10
10/10 [==============================] - 0s 483us/sample - loss: 666.6307
Epoch 6/10
10/10 [==============================] - 0s 707us/sample - loss: 877.8005
Epoch 7/10
10/10 [==============================] - 0s 611us/sample - loss: 580.0933
Epoch 8/10
10/10 [==============================] - 0s 693us/sample - loss: 1129.6672
Epoch 9/10
10/10 [==============================] - 0s 753us/sample - loss: 948.7717
Epoch 10/10
10/10 [==============================] - 0s 786us/sample - loss: 719.8773
